# Import all necessary modules

In [5]:
# import all neceassary modules
import sys
from Experimentation.monte_carlo import *
from Experimentation.ensemble_models import *
from Experimentation.get_alpha import *
from Experimentation.quantile import *
from Experimentation.weighting_scheme import *
from Experimentation.wis_score import *
from Experimentation.model_averaging import *
import numpy as np
import pandas as pd
import random
random.seed(42)
np.random.seed(42)

In [6]:
filename = 'data/alpha.txt'
alphas = get_alpha(filename)

state = 'New York' #if use Monroe or SantaRosa for Monroe or Santa Rosa county

if state=='New York':
    s_id = '36'
elif state=='Monroe':
    s_id='Monroe'
elif state=='Santa Rosa':
    s_id='SantaRosa'

# Ensemble Weights Calculation

Weights are calculated based on the formula used in paper.

In [4]:

fname = 'data/all_methods_state_{}.csv'.format(51)
data = pd.read_csv(fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
# data = data[data.forecast_date<pd.to_datetime('2021-01-01')]

# phases = pd.read_csv('NY_phts_ind_2022-01-15.csv', parse_dates=['target_end_date','V1'])
# phases['V1'] = phases['V1'] + pd.Timedelta(days=1) 

data = model_weights(data, t_p=4)
data = change_weights_format(data)
data.to_csv('weights/weights_'+fname.split('/')[-1]) ## creates a new file containing weights

KeyboardInterrupt: 

# Ensemble Forecast using LOP, Vincent, Mean, Median averaging

In [13]:
fname = 'weights/phase_weights_all_methods_state_{}.csv'.format(s_id)
weights = pd.read_csv(fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
f_fname = 'all_methods_state_{}.csv'.format(s_id)
data = pd.read_csv(f_fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
out = Vincent_Ensemble(data, weights, alphas)  ## for vincent use Vincent_Ensemble, for mean use Mean_Ensemble, and for median use Median_Ensemble
out.to_csv('Vincent-Ensemble_{}.csv'.format(s_id))

# Calculate WIS

In [20]:
import glob
for file in glob.glob('mean-median/*'):
    
    # fname = 'Vincent-Ensemble_{}_phases.csv'.format(s_id) #filename to get WIS score
    fname = file.split('/')[-1]
    data = pd.read_csv(file, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
    data = get_all_wis_score(data, alphas)
    data.to_csv(file)

In [5]:
###get median
for file in glob.glob('wis-ensemble/*'):
    fname = file.split('/')[-1].split('.')[0]
# fname = 'Vincent-Ensemble_{}.csv'.format(s_id) #filename to get WIS score
    data = pd.read_csv(file, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
    data = get_all_wis_score(data, alphas)
    data['mape'] = np.abs(data['q_0.5']-data['cases'])/data['cases'] 
    data['rwis'] = data['wis']/data['cases']
    data = data[['forecast_date','target_end_date','target','method','wis','mape','FIPS','rwis']]
    data = data.groupby(['forecast_date','target_end_date','target','method']).median().reset_index()
    data = data.rename(columns={'method':'model'})
    data.to_csv(f'Analysis/ensemble/{fname}_median.csv')

/tmp/ipykernel_474563/2397931273.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(['forecast_date','target_end_date','target','method']).median().reset_index()
/tmp/ipykernel_474563/2397931273.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(['forecast_date','target_end_date','target','method']).median().reset_index()
/tmp/ipykernel_474563/2397931273.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns whi

In [12]:
# Mean and MedianAnalysis/mean-median/

In [19]:
import glob
import pandas as pd
from multiprocessing import Pool

# Define a function to process each file
def process_file(fname):
    s_id = fname.split('/')[-1].split('.')[0].split('_')[-1]
    f_fname = 'data/all_methods_state_{}.csv'.format(s_id)
    data = pd.read_csv(f_fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
    
    quantiles = [f'q_{q}' for q in alphas]
    
    mean = pd.DataFrame()
    median = pd.DataFrame()
    for (f_date, t_date, fips,location, target),group in data.groupby(['forecast_date','target_end_date','FIPS','location','target']):
        mean_ensemble = group[quantiles].mean().to_dict()
        mean_ensemble['model'] = 'Mean-Ensemble'
        mean_ensemble['forecast_date'] = f_date
        mean_ensemble['FIPS'] = fips
        mean_ensemble['location'] = location
        mean_ensemble['target_end_date'] = t_date
        mean_ensemble['target'] = target
        mean_ensemble['point'] = mean_ensemble['q_0.5']
        mean_ensemble['cases'] = group['cases'].max()
        mean_ensemble = pd.DataFrame([mean_ensemble])
        median_ensemble = group[quantiles].median().to_dict()
        median_ensemble['model'] = 'Median-Ensemble'
        median_ensemble['forecast_date'] = f_date
        median_ensemble['FIPS'] = fips
        median_ensemble['location'] = location
        median_ensemble['target_end_date'] = t_date
        median_ensemble['target'] = target
        median_ensemble['point'] = median_ensemble['q_0.5']
        median_ensemble['cases'] = group['cases'].max()
        median_ensemble = pd.DataFrame([median_ensemble])

        mean = pd.concat([mean, mean_ensemble], ignore_index=True)
        median = pd.concat([median, median_ensemble], ignore_index=True)

    mean.to_csv(f'mean-median/mean-ensemble_{s_id}.csv')
    median.to_csv(f'mean-median/median-ensemble_{s_id}.csv')

if __name__ == '__main__':
    # Get the list of file names
    files = glob.glob('weights/*')
    
    # Define the number of processes (adjust as needed)
    num_processes = 12
    
    # Use a Pool of processes to parallelize the execution
    with Pool(num_processes) as pool:
        pool.map(process_file, files)


In [42]:
# Uva ensemble

In [59]:
data = pd.read_csv('forecasts-UVA-ensemble_quantiles.csv',parse_dates=['target_end_date','forecast_date'], dtype={'FIPS':'str'}, index_col=0)
data['location'] = data['FIPS'].apply(lambda x: x[:2])
data.to_csv('UVA-ensemble.csv')

In [60]:
data = get_all_wis_score(data, alphas)
data['mape'] = np.abs(data['q_0.5']-data['cases'])/data['cases'] 
data['rwis'] = data['wis']/data['cases']
data = data[['forecast_date','target_end_date','target','model','wis','mape','FIPS','rwis','location']]
    
for location, group in data.groupby('location'):
    # data = pd.read_csv(file, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
    group = group.groupby(['forecast_date','target_end_date','target','model']).median().reset_index()
    # group = group.rename(columns={'method':'model'})
    group.to_csv(f'Analysis/uva/UVA-ensemble_{location}_median.csv')

/tmp/ipykernel_416815/2939826160.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group = group.groupby(['forecast_date','target_end_date','target','model']).median().reset_index()
/tmp/ipykernel_416815/2939826160.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group = group.groupby(['forecast_date','target_end_date','target','model']).median().reset_index()
/tmp/ipykernel_416815/2939826160.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns whic

In [56]:
data

,forecast_date,target,target_end_date,FIPS,q_0.025,q_0.1,q_0.25,q_0.5,q_0.75,q_0.9,q_0.975,model,cases,location,mape,wis,rwis
0,2020-08-03,1 wk ahead inc case,2020-08-08,51001,0.000000,0.000000,5.841837,27.380563,48.919288,68.304813,89.968803,UVA-ensemble,18.0,51,0.521142,7.011242,0.389513
1,2020-08-03,1 wk ahead inc case,2020-08-08,51003,0.000000,11.424183,39.724830,71.168904,102.612978,130.913625,162.540570,UVA-ensemble,92.0,51,0.226425,12.042870,0.130901
2,2020-08-03,1 wk ahead inc case,2020-08-08,51005,2.295848,3.124934,3.866823,4.691115,5.515407,6.257296,7.086382,UVA-ensemble,6.0,51,0.218148,0.566909,0.094485
3,2020-08-03,1 wk ahead inc case,2020-08-08,51007,0.000000,0.212726,4.260032,8.756884,13.253735,17.301042,21.824046,UVA-ensemble,6.0,51,0.459481,1.680372,0.280062
4,2020-08-03,1 wk ahead inc case,2020-08-08,51009,0.000000,9.650274,20.350061,32.238302,44.126543,54.826331,66.783711,UVA-ensemble,57.0,51,0.434416,11.302655,0.198292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853035,2022-01-03,1 wk ahead inc case,2022-01-08,56043,0.000000,0.000000,2.507961,7.361670,12.215378,16.583868,21.465805,UVA-ensemble,35.0,56,0.789667,20.907429,0.597355
853036,2021-12-13,4 wk ahead inc case,2022-01-08,56045,20.928441,20.947208,20.964002,20.982661,21.001319,21.018113,21.036880,UVA-ensemble,9.0,56,1.331407,11.957175,1.328575
853037,2021-12-20,3 wk ahead inc case,2022-01-08,56045,16.051504,22.853802,28.940683,35.703650,42.466617,48.553498,55.355796,UVA-ensemble,9.0,56,2.967072,17.466250,1.940694
853038,2021-12-27,2 wk ahead inc case,2022-01-08,56045,0.000000,0.000000,0.000000,12.334193,26.555983,39.356038,53.660539,UVA-ensemble,9.0,56,0.370466,3.880917,0.431213


In [13]:
fname = 'forecasts-UVA-ensemble_quantiles.csv'
data = pd.read_csv(fname,dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
# data_uva = data.drop_duplicates(['forecast_date','target'])
data_uva = get_all_wis_score(data, alphas)
data_uva['target'] = data_uva['target'].apply(lambda x: x.split(' ')[0]+'-step_ahead')
data_uva['point'] = data_uva['q_0.5']
data_uva.to_csv('UVA-ensemble-all-location.csv')

In [14]:
for fname in glob.glob('wis-ensemble/*'):

    # fname = 'Analysis/mean-median/mean-ensemble_{}_median.csv'.format(state)
    data = pd.read_csv(fname,dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
    data = data.drop_duplicates(['forecast_date','target'])
    data['model'] = data['method']
    data = get_all_wis_score(data, alphas)
    data.to_csv(fname)